# Goldstein-price function

**Contributed by**: Benoît Legat

In this example, we consider the minimization of the [Goldstein-price function](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

In [1]:
using SumOfSquares
using DynamicPolynomials

Create *symbolic* variables (not JuMP *decision* variables)

In [2]:
@polyvar x[1:2]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂],)

To use Sum-of-Squares Programming, we first need to pick an SDP solver,
see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [3]:
import Clarabel
using Dualization
model = SOSModel(dual_optimizer(Clarabel.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Dual model with Clarabel attached

Create a JuMP decision variable for the lower bound

In [4]:
@variable(model, γ)

γ

`f(x)` is the Goldstein-Price function

In [5]:
f1 = x[1] + x[2] + 1
f2 = 19 - 14*x[1] + 3*x[1]^2 - 14*x[2] + 6*x[1]*x[2] + 3*x[2]^2
f3 = 2*x[1] - 3*x[2]
f4 = 18 - 32*x[1] + 12*x[1]^2 + 48*x[2] - 36*x[1]*x[2] + 27*x[2]^2
f = (1 + f1^2*f2) * (30 + f3^2*f4)

600 + 720x₂ + 720x₁ + 3060x₂² - 4680x₁x₂ + 1260x₁² + 12288x₂³ - 19296x₁x₂² + 7344x₁²x₂ - 1072x₁³ + 14346x₂⁴ - 23616x₁x₂³ + 7776x₁²x₂² + 5784x₁³x₂ - 2454x₁⁴ + 1944x₂⁵ - 11880x₁x₂⁴ + 5040x₁²x₂³ + 9840x₁³x₂² - 7680x₁⁴x₂ + 1344x₁⁵ - 4428x₂⁶ - 1188x₁x₂⁵ + 8730x₁²x₂⁴ + 1240x₁³x₂³ - 5370x₁⁴x₂² - 168x₁⁵x₂ + 952x₁⁶ - 648x₂⁷ + 1944x₁x₂⁶ + 3672x₁²x₂⁵ - 3480x₁³x₂⁴ - 4080x₁⁴x₂³ + 2592x₁⁵x₂² + 1344x₁⁶x₂ - 768x₁⁷ + 729x₂⁸ + 972x₁x₂⁷ - 1458x₁²x₂⁶ - 1836x₁³x₂⁵ + 1305x₁⁴x₂⁴ + 1224x₁⁵x₂³ - 648x₁⁶x₂² - 288x₁⁷x₂ + 144x₁⁸

Constraints `f(x) - γ` to be a sum of squares

In [6]:
con_ref = @constraint(model, f >= γ)
@objective(model, Max, γ)
optimize!(model)

-------------------------------------------------------------
           Clarabel.jl v0.5.1  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 45
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 121
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : PSDTriangle = 1,  numel = 120

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       ga

The lower bound found is 3

In [7]:
solution_summary(model)

* Solver : Dual model with Clarabel attached

* Status
  Result count       : 1
  Termination status : NUMERICAL_ERROR
  Message from the solver:
  "NUMERICAL_ERROR"

* Candidate solution (result #1)
  Primal status      : OTHER_RESULT_STATUS
  Dual status        : OTHER_RESULT_STATUS
  Objective value    : 3.00063e+00
  Dual objective value : 3.00047e+00

* Work counters
  Solve time (sec)   : 3.02388e-01
  Barrier iterations : 14


The moment matrix is as follows, we can already see the global minimizer
`[0, -1]` from the entries `(2, 1)` and `(3, 1)`.
This heuristic way to obtain solutions to the polynomial optimization problem
is suggested in [L09, (6.15)].

[L09] Laurent, Monique.
*Sums of squares, moment matrices and optimization over polynomials.*
Emerging applications of algebraic geometry (2009): 157-270.

In [8]:
ν = moment_matrix(con_ref)

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3, x[2]^4, x[1]*x[2]^3, x[1]^2*x[2]^2, x[1]^3*x[2], x[1]^4])
And entries in a 15×15 SymMatrix{Float64}:
  1.0000000024148903     -0.9999382587027277     …   3.5515359712023664e-5
 -0.9999382587027277      0.9998826624220426         1.5428589596678468e-5
  1.3387620375239558e-5  -1.0749343383504728e-5      7.168403374125203e-5
  0.9998827366832189     -0.9998262504021791         0.0007786025878914721
 -1.064465003841754e-5    1.3545750928683232e-5      0.0007646370434974884
  9.681349389043938e-6    1.926321643119007e-6   …   0.001679359987193148
 -0.999826388103798       0.9997725258114581        -0.0005196476259947733
  1.3414924328496838e-5  -1.1292688444729286e-5     -0.0002647962832027735
  1.9048022059954632e-6   3.770656344920049e-6       0.001080269454981368
  1.2957759703460782e-5   8.943719503824193e-6       0.0026021410399755048
  0.999772653165

Many entries of the matrix actually have the same moment.
We can obtain the following list of these moments without duplicates
(ignoring when difference of entries representing the same moments is below `1e-5`)

In [9]:
μ = measure(ν, atol = 1e-5)

Measure{Float64, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}([1.0000000024148903, -0.9999382587027277, 1.3387620375239558e-5, 0.9998827366832189, -1.064465003841754e-5, 9.681349389043938e-6, -0.999826388103798, 1.3414924328496838e-5, 1.9048022059954632e-6, 1.2957759703460782e-5  …  0.0026021410399755048, 1.494070202278145, -0.4025431439050152, 0.5365928440702569, -0.33638361424492635, 0.6335530365500794, -0.1904128211623446, 0.853947475960679, 0.13972872187918567, 1.3573135696530392], DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[1, x₂, x₁, x₂², x₁x₂, x₁², x₂³, x₁x₂², x₁²x₂, x₁³  …  x₁⁷, x₂⁸, x₁

The truncated moment matrix can then be obtained as follows

In [10]:
ν_truncated = moment_matrix(μ, monomials(x, 0:3))

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3])
And entries in a 10×10 SymMatrix{Float64}:
  1.0000000024148903     -0.9999382587027277     …  1.2957759703460782e-5
 -0.9999382587027277      0.9998827366832189        8.947452650547174e-6
  1.3387620375239558e-5  -1.064465003841754e-5      3.5515359712023664e-5
  0.9998827366832189     -0.999826388103798         2.2621613918790728e-6
 -1.064465003841754e-5    1.3414924328496838e-5     1.5428589596678468e-5
  9.681349389043938e-6    1.9048022059954632e-6  …  7.168403374125203e-5
 -0.999826388103798       0.9997726531656117        0.0002347760801025644
  1.3414924328496838e-5  -1.1329709386179011e-5     0.0007786025878914721
  1.9048022059954632e-6   3.7294997814120735e-6     0.0007646370434974884
  1.2957759703460782e-5   8.947452650547174e-6      0.001679359987193148

Let's check if the flatness property is satisfied.
The rank of `ν_truncated` seems to be 1:

In [11]:
using LinearAlgebra
LinearAlgebra.svdvals(Matrix(ν_truncated.Q))
LinearAlgebra.rank(Matrix(ν_truncated.Q), rtol = 1e-3)
svdvals(Matrix(ν_truncated.Q))

10-element Vector{Float64}:
 3.9993858652025946
 0.002487503363528851
 0.0006002562669328149
 7.234559265742689e-5
 7.299494866960483e-6
 4.6270547394527895e-6
 1.5969044347469254e-6
 7.048918314978201e-7
 1.5419035101956763e-7
 4.5447029262940007e-8

The rank of `ν` is clearly higher than 1, closer to 3:

In [12]:
svdvals(Matrix(ν.Q))

15-element Vector{Float64}:
 5.181337008500714
 2.315075382581048
 1.3721771781426986
 0.006399411637137132
 0.0020867026437841546
 0.0004508445856077911
 0.0004137758357998113
 8.274359875909834e-5
 8.1559894746372e-6
 4.24788408956209e-6
 1.4377755328764997e-6
 8.091189698697273e-7
 2.1174067670630372e-7
 9.080062313748886e-8
 4.868239852446525e-9

Even if the flatness property is not satisfied, we can
still try extracting the minimizer with a low rank decomposition of rank 3.
We find the optimal solution again doing so:

In [13]:
atomic_measure(ν, FixedRank(3))

Atomic measure on the variables x[1], x[2] with 1 atoms:
 at [1.511543675655948e-5, -0.9999415621371347] with weight 1.033075101614307

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*